In [9]:
#import statements
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

#from keypass import NOAA_api
import pylab as pl
from IPython import display

from functools import partial
import pyproj
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score
from numpy import absolute,mean,std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive
from IPython.display import Image
from IPython.core.display import HTML 

# <div class="alert alert-block alert-info"><span style='color: Black'>Modeling CO2 Sequestration using the Global Ocean Data Analysis Project (GLODAP)</span>
[GitHub Repo](https://github.com/ossana1/DATA606_FinalProject) | [Project Site](https://sites.google.com/s/14-zXY-tR-4ddTR09NcwHH0VdedwqjA0Q/p/1mnouXuUqS3ud_088LqPGmpKfiSUgTiD3/edit?ths=true)
## Project Goal: Carbon Dioxide Concentration Modeling to Fill GLODAP Dataset Gaps
The GLODAP v2.2020 dataset has approximately 65% of entries missing $tCO_2$values. The goal of this project is to use a regression (machine learning) ML model to fill the missing entries in the dataset, using measurements of pH, alkalinity, geolocation, temperature, date, etc. as model inputs. The data set was split into training, test, and validation sets using rows not missing $tCO_2$ values. After training and testing regression ML models, the best performing model was applied to the rows missing $tCO_2$ values. 

Increasing the robustness of this data set could help predict future net carbon dioxide absorption areas which may be at higher risk for earlier ocean acidification than predicted. It may be possible to protect these areas in ways that slow or mitigate the changes in TCO2  that can damage marine life and a valuable food source for mankind. <br>

### <span style='color: Purple'>Figures 1 and 2 use a condensed version of the data in the data set after the $tCO_2$ missing values are filled with a machine learning, random forest model.</span>

Every 25th row is saved to make the dashboard useable (ie faster). There were 1218966 rows before the trim and 48759 after. There are 33 cruises with <100 datapoints out of 936 cruises, this data loss is acceptable to make the dashboard respond to users in real time. 

### Model Performance: filling empty carbon dioxide concentration values

Model performance on train, test, and validation sets for $tCO_2$ ML random forest regression model ($R^2$ and explained variance of ~0.99 on all sets). The mean square error of the train, test, validation sets were 9.84, 77.0, 68.7, respectively.
<img src="https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/images/MLperformance.png"> 

### Ocean region approxiamations using rectangular polygons via geopandas.
<img src="https://github.com/ossana1/DATA606_FinalProject/blob/master/images/OceanBoundgeo.png?raw=true">

### <div class="alert alert-block alert-info"><span style='color: Black'> Figure 1. Historical data after filling missing $tCO_2$ values. </span>

### <div class="alert alert-block alert-info"><span style='color: Black'> Figure 2. Historical data after filling missing $tCO_2$ values: 3D depth plot to see variable changes with depth, latitude, and longitude. </span>

In [10]:
#https://stackoverflow.com/questions/60609760/is-there-a-way-to-import-csv-file-from-github-automatically-to-my-jupyter-notebo (accessed Oct. 08, 2020).

#df =pd.read_csv('C:\\Users\\ossan\\DATA606\\DATA\\TCO2_filled.csv',index_col=0) #ORIGINAL
#plotly is too slow with complete set -use this for now 
url = 'https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/data/TCO2_small.csv'
df = pd.read_csv(url,index_col=0)
dftrim = df 
#print(len(df))
df['year'] =pd.to_numeric( df['year'])
df['year'] =df['year'].astype(int)
g=df.groupby('year')

In [11]:
frames = {} # make dictionary of dataframes per year
for x,y in g:
    frames[x] = y
#frames[list(frames.keys())[-1]]

In [12]:
#make variable list for viusla 
var = df.columns 
years = list(frames.keys())
list_updatemenus=[];temp_dict={}
#make dictionaries for dropdowns for visual
for n, year in enumerate(years):
    visible = [False] * len(years)
    visible[n] = True
    temp_dict = dict(label = str(year),method = 'update',
                 args = [{'visible': visible},{'title': 'Year %d' % year}])
    list_updatemenus.append(temp_dict)
    
list_c=[];c_dict={}
for n, year in enumerate(var):
    visible = [False] * len(var)
    visible[n] = True
    c_dict = dict(label = str(var),method = 'update',
                 args = [{'visible': visible}, {'title': var}])
    list_c.append(c_dict)
    

In [13]:
#make list for nicer formatted columns for visuals 
col=['Cruise', 'Station',  'Year', 'Month', 'Day', 'Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth',
       'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'Oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']

actu=['cruise', 'station', 'year', 'month', 'day', 'bottomdepth', 'maxsampdepth', 'pressure', 'depth',
       'temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']
unit=[ 'Cruise ID', 'm', 'm','bar', 'm', 'C', 'C', 'g/kg', 'kg m-3',
       'kg m-3', 'kg m-3', 'kg m-3', 'kg m-3', 'umol kg-1', 'umol kg-1', 'umol kg-1','umol kg-1', 
      'umol kg-1', 'umol kg-1', 'umol kg-1', 'pH', 'pH','umol kg-1']

def plot_compare_lag(Variable, Year):
    #make figure 
    var = Variable; year = Year
    fig5 =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=dftrim["longitude"],
                                  lat=dftrim["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 2, color = dftrim[actu[col.index(var)]], colorscale = 'Inferno',
            cmax = df[actu[col.index(var)]].max(),
        colorbar = dict( ticks = "outside",title=unit[col.index(var)],ypad=0,len=.8
        ))),

    layout= dict(title ='<b>Historical Data- Variable: ' +var + ' | Year:'  + str(year.astype(str)[0:4]) +'</b>')  )

    fig5.show()
    return fig5 
interactive(plot_compare_lag, Variable=col, 
            Year=list(np.linspace(1984,2019,(2019-1983))),  )


interactive(children=(Dropdown(description='Variable', options=('Cruise', 'Station', 'Year', 'Month', 'Day', '…

In [14]:
cold=[   'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'Oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']

actud=['temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']
unitd=['C', 'C', 'g/kg', 'kg m-3',
       'kg m-3', 'kg m-3', 'kg m-3', 'kg m-3', 'umol kg-1', 'umol kg-1', 'umol kg-1','umol kg-1', 
      'umol kg-1', 'umol kg-1', 'umol kg-1', 'pH', 'pH','umol kg-1']

def depthfigure(Variable,Year):
    year = Year;    var = Variable
    dfd= dftrim[dftrim['year'] ==year]
    v = actu[cold.index(var)]
    figdepth = go.Figure(data=[go.Scatter3d( x=dfd.longitude,y=dfd.latitude,z=dfd.depth,
        mode='markers',
        marker=dict(colorbar = dict( ticks = "outside",title=unitd[cold.index(var)],ypad=0,len=.8),
            size=3,
            color=dfd[v],# set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8) )])
    
    figdepth.update_layout(title_text='<b>Depth Plot: '+ var+'</b>' +' '+str(year)[0:4],
                          scene=dict(xaxis=dict(title='Longitude'),yaxis=dict(title='Latitude'),
                                    zaxis=dict(title='Depth (m)')))

    figdepth.show()
    return figdepth

interactive(depthfigure,Variable=cold,Year=list(np.linspace(1984,2019,(2019-1983))), )


interactive(children=(Dropdown(description='Variable', options=('Potential Temperature', 'theta', 'Salinity', …

***

### <div class="alert alert-block alert-info"><span style='color: Black'> Figure 3. Percent change of a variable over a user defined time range.</span>

The complete (filled) dataset is first grouped by latitude and longitude and averaged per year. The latitude and longitude of the ending year in the query is used with sklearn BallTree to find (haversine distance) the closest cruise geolocation in the startinng year. The percent difference between the starting year and end year variable of choice is displayed (end year- start year value). One shortcoming is that depth of the samples are not taken into account at this time.

**Temperature was removed because the values are an average over a year and do not consider depth or season. The large percent changes in O2 concentration and oxygen utilization are likely for the same reasons.**


In [15]:
url = 'https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/data/Groupedlatlongyr.csv'
latlong= pd.read_csv(url,index_col=0)
latlong.drop(columns=['cruise','station','day','month','Fill','cast'],inplace=True)
#clean up columns for labels 
colu=['Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'Oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']
actua=[  'salinity', 'sigma0','sigma1', 'sigma2', 'sigma3', 'gamma', 'oxygen', 'aou',
       'nitrate','nitrite', 'silicate', 'phosphate', 'talk', 'phts25p0', 'phtsinsitutp','tco2']

unit2=[  'g/kg', 'kg m-3','kg m-3', 'kg m-3', 'kg m-3', 'kg m-3', 'umol kg-1', 'umol kg-1', 
      'umol kg-1','umol kg-1', 'umol kg-1', 'umol kg-1', 'umol kg-1', 'pH', 'pH','umol kg-1']
%matplotlib inline

from sklearn.neighbors import BallTree

def chang(var,start,end,df):
    start_year=latlong[latlong.year ==start].reset_index(); end_year=latlong[latlong.year ==end].reset_index()
    query_lats = end_year.latitude;query_lons = end_year.longitude
    
    varx = actua[colu.index(var)] 
    #search for closest lat and long 
    ###https://stackoverflow.com/questions/10549402/kdtree-for-longitude-latitude
    bt = BallTree(np.deg2rad(start_year[['latitude', 'longitude']].values), metric='haversine')
    distances, indices = bt.query(np.deg2rad(np.c_[query_lats, query_lons]))

    i = pd.DataFrame(list(zip(distances,indices)),columns=['distances','indices'])
    end_year['compare']  =i.iloc[:,1].astype(int)
    end_year['data'] = start_year.loc[end_year['compare'],varx].values
    #calculate the percent change (x2-x1)/x1
    end_year['diff'] =(end_year[varx]- end_year['data'])/end_year.data*100
    return end_year

def plot_compare_lag(Variable, StartYear,EndYear):
    var = Variable ;startyear = StartYear;endyear = EndYear 
    change = chang(var,startyear,endyear,latlong)
    fig =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=change["longitude"],
                                  lat=change["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 4, color = change['diff'], colorscale = 'Inferno',
        colorbar = dict( ticks = "outside",title='%Change' +' '+unit2[colu.index(var)],ypad=0,len=.8
                       ) )), #cmax = 1.5*change['diff'].max(),
    layout= dict(title ='<b>Variable % Change: ' +var + ' | Start Year:'  + 
                 str(startyear.astype(str)[0:4]) +'| End Year:'  + 
                 str(endyear.astype(str)[0:4]) +'</b>') ,)
    fig.show()
    return fig 



interactive(plot_compare_lag, Variable=colu, StartYear=list(np.linspace(1984,2019,(2019-1983))), 
            EndYear=list(np.linspace(1984,2019,(2019-1983))),)


interactive(children=(Dropdown(description='Variable', options=('Salinity', 'Potential Temp 0m', 'Potential Te…

***

***

### <div class="alert alert-block alert-info"><span style='color: Black'> Figure 4. See the percent change every 1 or 4 years as a boxplot. </span>
The complete (filled) dataset is first grouped by latitude and longitude and averaged per year. The latitude and longitude of the ending year in the query is used with sklearn BallTree to find (haversine distance) the closest cruise geolocation in the startinng year. The percent difference between the starting year and end year variable. The bouding end year is set as 2018 due to the limited 2019 data.

**The dataset can be grouped by the major ocean regions, the ocean biomes identified in the introduction, or by latitude range.**

In [16]:
delta4yr='https://github.com/ossana1/DATA606_FinalProject/blob/master/data/ChangeEvery4years.xlsx?raw=true'
deltayr='https://github.com/ossana1/DATA606_FinalProject/blob/master/data/ChangeYeartoyear.xlsx?raw=true'
df4= pd.read_excel(delta4yr);df1= pd.read_excel(deltayr)
deltas=[ 'tco2diff','salinitydiff','gammadiff',
       'oxygendiff', 'phosphatediff', 'talkdiff', 'phts25p0diff', 'phtsinsitutpdiff']

realname=['TCO2','Salinity','Gamma','Oxygen','Phosphate','Total Alkalinity',
         'pH @ STP','pH in Situ']

def plot_box(Variable,Hue,Interval):
    #make figure
    if Interval =='4yrs':
        df =df4; a = '4'
    else:
        df = df1; a='1'
    var = deltas[realname.index(Variable)]
    if Hue =='Major Ocean Region':
        c = 'major'
    elif Hue =='Latitude Region': 
        c = 'Latitude Range'
    else:
        c = 'Region'
    
    box=go.Figure(px.box(df, x='Year', y=var,color=c))
    box.update_yaxes(title_text='Percent Change for ' +realname[deltas.index(var)])
    box.update_xaxes(title_text='Year Range')
    box.update_layout(title_text='<b>Percent Change Every '+a+' Years: ' +realname[deltas.index(var)]+'</b>')
    box.show()
    return box


interactive(plot_box, Variable=realname, Interval=['4yrs','1yrs'],Hue = ['Major Ocean Region',
                        'Latitude Region','Detailed Ocean Regions'])



interactive(children=(Dropdown(description='Variable', options=('TCO2', 'Salinity', 'Gamma', 'Oxygen', 'Phosph…